'''
THIS FILE IS FOR TRAINING THE TREE PHENOLOGY MODEL
CODE WRITTEN FOR CUBESAT MANTIS PROGRAM (DSS)
AUTHOR: CHAZ DAVIES
''' 


In [1]:

import tensorflow as tf
import numpy as np
import matplotlib as img 
import rasterio
import os
import image_process as ip
from rasterio.plot import show
import json
import keras


In [2]:

path = 'C:\\Users\\chazd\\OneDrive\Desktop\\git\DSS\DSS-MANTIS-OBC-AI\\Images\\tree-phen\\trees_feb2020-Dec2021_10m'

In [3]:
## set constants
# might change during training process
image_height = 160
image_width = 175
bands = 4


In [4]:
# Make array to store label names

labels = {
        "evergreen-coverage" : 0, # when deciduous trees had no leaves
        "light-canopy" : 1, # when leaves are starting to form
        "dense-canopy" : 2, # when the trees are at their greenest
        "start-of-senescence" : 3, # when leaves begin to change colour,
        "senescence" : 4# when the leaves are very colourful
    }


labels

{'evergreen-coverage': 0,
 'light-canopy': 1,
 'dense-canopy': 2,
 'start-of-senescence': 3,
 'senescence': 4}

In [5]:
# copy the path
tiff_files = ip.loadDataSet(path)
file_path = ["" for x in range(len(tiff_files))]

# concactinate the folder path to the file name
for i in range(len(tiff_files)):
    file_path[i] = os.path.join(path, tiff_files[i])


In [6]:
#extract dates
dates = ["" for x in range(len(file_path))]
for i, file in enumerate(file_path):
    dates[i] = file[len(file)-15:len(file)-5]



In [7]:
raster_imgs = np.empty(len(file_path)) 

#open each file
img = rasterio.open(file_path[1])

    


c:\Users\chazd\AppData\Local\Programs\Python\Python39\lib\site-packages\rasterio\__init__.py:331: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [8]:
# Open images and move them to a numpy array
batch = len(file_path)
# set dimension of 3d-array to image height, width and number of bands
# ---- maybe add a dimension for labels

unlabeled_images = np.zeros((batch, image_height, image_width, bands))

for i in range(batch):
    #open raster image
    img = rasterio.open(file_path[i])

    for j in range(bands):
        unlabeled_images[i,:,:,j] = img.read(j+1)



In [9]:
# Load JSON file with image file names and their corresponding labels
with open('C:\\Users\\chazd\\OneDrive\\Desktop\\git\\DSS\\DSS-MANTIS-OBC-AI\\Images\\trees_feb2020-Dec2021_10m_labels.json') as json_file:
    data = json.load(json_file)

# Extract filenames and annotations
file_names = [item["filename"] for item in data]
annotations_from_json = [item["annotations"] for item in data]

# Map annotations from JSON to numerical labels using the label dictionary
numerical_labels = np.array([labels[annotation] for annotation in annotations_from_json])


## Preprocessing

Before prepocessing the images, a quick review of the dataset

In [10]:
print("Dimensions: ", unlabeled_images.shape)
print("Batch Size: ", len(unlabeled_images))

Dimensions:  (66, 160, 175, 4)
Batch Size:  66


In [11]:
# normalize all values
import cv2
import numpy as np

def normalize_4channel_image(image):
    # Split the image into Red, Green, Blue, and NIR channels
    r, g, b, nir = cv2.split(image)

    # Normalize each channel
    r_normalized = (r - np.min(r)) / (np.max(r) - np.min(r))
    g_normalized = (g - np.min(g)) / (np.max(g) - np.min(g))
    b_normalized = (b - np.min(b)) / (np.max(b) - np.min(b))
    nir_normalized = (nir - np.min(nir)) / (np.max(nir) - np.min(nir))

    # Recombine the channels
    normalized_image = cv2.merge([r_normalized, g_normalized, b_normalized, nir_normalized])

    return normalized_image



In [21]:
normalized= unlabeled_images

for i in range(len(unlabeled_images)):
    
    normalized[i] = normalize_4channel_image(unlabeled_images[i])
    


In [24]:
#split into test and train
split = np.random.randint(0,len(normalized), size = int(len(normalized)*0.9))

train_images = normalized[split]
train_labels = numerical_labels[split]
test_images = normalized[-split]
test_labels = numerical_labels[-split]


## Building Model

In [18]:
from tensorflow.keras import datasets, layers, models

In [19]:
model = models.Sequential()

model.add(layers.Conv2D(16,(3,3),1,activation= 'relu', input_shape=(160,175,4)))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(32,(3,3),1, activation= 'relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(16,(3,3),1, activation= 'relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5))

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 158, 173, 16)      592       
                                                                 
 max_pooling2d (MaxPooling2  (None, 79, 86, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 77, 84, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 38, 42, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 36, 40, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 18, 20, 16)       

In [20]:
#compile
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



## Train Model

In [25]:
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Epoch 1/10


2/2 [==============================] - 4s 722ms/step - loss: 1.3172 - accuracy: 0.6949 - val_loss: 1.6440 - val_accuracy: 0.5424
Epoch 2/10
2/2 [==============================] - 0s 268ms/step - loss: 1.1381 - accuracy: 0.6949 - val_loss: 1.3354 - val_accuracy: 0.5424
Epoch 3/10
2/2 [==============================] - 0s 274ms/step - loss: 0.9741 - accuracy: 0.6949 - val_loss: 1.2029 - val_accuracy: 0.5424
Epoch 4/10
2/2 [==============================] - 0s 274ms/step - loss: 0.9124 - accuracy: 0.6949 - val_loss: 1.1836 - val_accuracy: 0.5424
Epoch 5/10
2/2 [==============================] - 0s 253ms/step - loss: 0.8484 - accuracy: 0.6949 - val_loss: 1.2228 - val_accuracy: 0.5424
Epoch 6/10
2/2 [==============================] - 0s 255ms/step - loss: 0.8235 - accuracy: 0.6949 - val_loss: 1.1503 - val_accuracy: 0.5424
Epoch 7/10
2/2 [==============================] - 0s 273ms/step - loss: 0.7846 - accuracy: 0.6949 - val_loss: 1.1065 - val_accuracy: 0.5424
Epoch 8/10
2/2 [==

## Model Evaluation

In [28]:
import matplotlib.pyplot as plt

# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)


2/2 - 0s - loss: 1.0452 - accuracy: 0.5424 - 144ms/epoch - 72ms/step


In [31]:
print("Accuracy = ", test_acc*100, "%")

Accuracy =  54.23728823661804 %
